In [1]:
import numpy as np
import random
import math

# Game Engine

In [2]:
class Game():
    def __init__(self):
        self.board = np.zeros([4, 4])
        self.last = None

    def new_piece(self):
        flattened_board = self.board.reshape(16)
        empty_spots = np.where(flattened_board == 0)[0]
        
        if len(empty_spots) == 0:
            return False
        else:
            new_spot = empty_spots[random.randint(0, len(empty_spots) - 1)]
            print(f"New cell created at {math.trunc(new_spot / 4)},{new_spot % 4}")
            flattened_board[new_spot] = 2 if random.randint(0, 10) else 4
            self.board = flattened_board.reshape(4, 4)

        return True


    def has_won(self):
        return np.any(self.board >= 2048) and not any(self.can_move(direction) for direction in ['L', 'R', 'U', 'D'])

    def has_lost(self):
        return not any(self.can_move(direction) for direction in ['L', 'R', 'U', 'D']) and not np.any(self.board >= 2048)

    def status(self):
        # Determine game status based on game conditions
        if any(self.can_move(direction) for direction in ['L', 'R', 'U', 'D']):
            return "Game Ongoing"
        elif self.has_won():
            return "Game Won"
        elif self.has_lost():
            return "Game Lost"
        else:
            return "Unknown State"
        
    def can_move(self, direction):
        if direction == 'L':  # Move left
            for row in self.board:
                for i in range(1, 4):
                    if row[i] != 0 and (row[i - 1] == 0 or row[i - 1] == row[i]):
                        return True
        elif direction == 'R':  # Move right
            for row in self.board:
                for i in range(2, -1, -1):
                    if row[i] != 0 and (row[i + 1] == 0 or row[i + 1] == row[i]):
                        return True
        elif direction == 'U':  # Move up
            for col in range(4):
                for row in range(1, 4):
                    if self.board[row][col] != 0 and (self.board[row - 1][col] == 0 or self.board[row - 1][col] == self.board[row][col]):
                        return True
        elif direction == 'D':  # Move down
            for col in range(4):
                for row in range(2, -1, -1):
                    if self.board[row][col] != 0 and (self.board[row + 1][col] == 0 or self.board[row + 1][col] == self.board[row][col]):
                        return True
        return False

    def transition_left(self):    
        for m in range(0, 4):
            merged = False # only allow one merge per row per turn, as per the rules
            
            for i in range(0, 4):
                for n in range(1, 4):
                    if not self.board[m][n] == 0 and not merged and self.board[m][n - 1] == self.board[m][n]:
                        print(f"Cells merged into {m},{n - 1}")
                        self.board[m][n - 1], self.board[m][n] = self.board[m][n - 1] * 2, 0
                        merged = True
                        
                    if self.board[m][n - 1] == 0 and self.board[m][n] != 0:
                        self.board[m][n - 1], self.board[m][n] = self.board[m][n], 0

    def transition_right(self):
        for m in range(0, 4):
            merged = False # only allow one merge per row per turn, as per the rules

            for i in range(0, 4):
                for n in range(2, -1, -1):
                    if not self.board[m][n] == 0 and not merged and self.board[m][n] == self.board[m][n + 1]:
                        print(f"Cells merged into {m},{n + 1}")
                        self.board[m][n], self.board[m][n + 1] = 0, self.board[m][n] * 2
                        merged = True
                    
                    if self.board[m][n] != 0 and self.board[m][n + 1] == 0:
                        self.board[m][n], self.board[m][n + 1] = 0, self.board[m][n]
            
    def transition_up(self):
        for n in range(0, 4):
            merged = False # only allow one merge per column per turn, as per the rules
            
            for i in range(0, 4):
                for m in range(1, 4):
                    if not self.board[m][n] == 0 and not merged and self.board[m - 1][n] == self.board[m][n]:
                        print(f"Cells merged into {m - 1},{n}")
                        self.board[m - 1][n], self.board[m][n] = self.board[m - 1][n] * 2, 0
                        merged = True
                        
                    if self.board[m - 1][n] == 0 and self.board[m][n] != 0:
                        self.board[m - 1][n], self.board[m][n] = self.board[m][n], 0

    def transition_down(self):
        for n in range(0, 4):
            merged = False # only allow one merge per column per turn, as per the rules

            for i in range(0, 4):
                for m in range(2, -1, -1):
                    if not self.board[m][n] == 0 and not merged and self.board[m][n] == self.board[m + 1][n]:
                        print(f"Cells merged into {m + 1},{n}")
                        self.board[m][n], self.board[m + 1][n] = 0, self.board[m][n] * 2
                        merged = True
                    
                    if self.board[m][n] != 0 and self.board[m + 1][n] == 0:
                        self.board[m][n], self.board[m + 1][n] = 0, self.board[m][n]
    
    def transition(self, direction):
        if self.can_move(direction):
            match direction:
                case 'L':
                    self.transition_left()
                case 'R':
                    self.transition_right()
                case 'U':
                    self.transition_up()
                case 'D':
                    self.transition_down()
            
            self.last = direction
            self.new_piece()          
                    

# Test new piece generation and basic movement

In [3]:
game = Game()
game.board

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [4]:
game.new_piece()
game.new_piece()
game.board

New cell created at 3,0
New cell created at 2,1


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 2., 0., 0.],
       [2., 0., 0., 0.]])

In [5]:
game.transition('L')
game.board

New cell created at 0,1


array([[0., 2., 0., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 0.],
       [2., 0., 0., 0.]])

In [6]:
game.transition('R')
game.board 

New cell created at 3,1


array([[0., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 2.],
       [0., 2., 0., 2.]])

# Test merge left

In [7]:
game = Game()
game.board[0][0], game.board[0][3] = 2, 2
game.board

array([[2., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [8]:
game.transition('L')
game.board

Cells merged into 0,0
New cell created at 0,3


array([[4., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# Test merge right

In [9]:
game = Game()
game.board[0][0], game.board[0][3] = 2, 2
game.board

array([[2., 0., 0., 2.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [10]:
game.transition('R')
game.board

Cells merged into 0,3
New cell created at 2,2


array([[0., 0., 0., 4.],
       [0., 0., 0., 0.],
       [0., 0., 2., 0.],
       [0., 0., 0., 0.]])

# Test merge up

In [11]:
game = Game()
game.board[0][0], game.board[3][0] = 2, 2
game.board

array([[2., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 0.]])

In [12]:
game.transition('U')
game.board

Cells merged into 0,0
New cell created at 1,1


array([[4., 0., 0., 0.],
       [0., 2., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

# Test merge down

In [13]:
game = Game()
game.board[0][0], game.board[3][0] = 2, 2
game.board

array([[2., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [2., 0., 0., 0.]])

In [14]:
game.transition('D')
game.board

Cells merged into 3,0
New cell created at 3,2


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [4., 0., 2., 0.]])